In [2]:
'''
Crear una red bayesiana desde un fichero bif
'''

from pgmpy.readwrite import BIFReader
from pgmpy.readwrite import BIFWriter
from pgmpy.inference import VariableElimination
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

reader = BIFReader("cancer.bif")

model = reader.get_model()

# Acceder a los elementos del modelo
var = reader.get_variables()
valores = reader.get_states()
edges = reader.get_edges()
cpds = reader.get_values()

print(f"Variables en el modelo: {var} \n")
print(f"Valores de la variables en el modelo: {valores} \n")
print(f"Aristas en el modelo: {edges} \n")
print(f"CPDs en el modelo: {cpds} \n")

# Imprimir el modelo en fichero bif

writer = BIFWriter(model)
writer.write_bif('cancerW.bif')

# Inferencia por eliminación de variables 

model_infer = VariableElimination(model)

# Calcular la probabilidad de Cancer dado Fumador = verdad

q = model_infer.query(variables=['Cancer'], evidence={'Fumador':'verdad'})
print(q)


Variables en el modelo: ['Polucion', 'Fumador', 'Cancer', 'Radiografia', 'Disnea'] 

Valores de la variables en el modelo: {'Polucion': ['baja', 'alta'], 'Fumador': ['verdad', 'falso'], 'Cancer': ['verdad', 'falso'], 'Radiografia': ['positiva', 'negativa'], 'Disnea': ['verdad', 'falso']} 

Aristas en el modelo: [['Polucion', 'Cancer'], ['Fumador', 'Cancer'], ['Cancer', 'Radiografia'], ['Cancer', 'Disnea']] 

CPDs en el modelo: {'Polucion': array([[0.9],
       [0.1]]), 'Fumador': array([[0.3],
       [0.7]]), 'Cancer': array([[0.03 , 0.001, 0.05 , 0.02 ],
       [0.97 , 0.999, 0.95 , 0.98 ]]), 'Radiografia': array([[0.9, 0.2],
       [0.1, 0.8]]), 'Disnea': array([[0.65, 0.3 ],
       [0.35, 0.7 ]])} 

+----------------+---------------+
| Cancer         |   phi(Cancer) |
+================+===============+
| Cancer(verdad) |        0.0320 |
+----------------+---------------+
| Cancer(falso)  |        0.9680 |
+----------------+---------------+


In [3]:
'''
Definir una red bayesiana en pgmpy supone los siguientes pasos:

Definir la estructura de la red
Definir los parámetros (CPDs)
Asociar los CPDs con la red. 
'''

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD


# Definir la estructura de la red

cancer_model = BayesianNetwork(
    [
        ("Polucion", "Cancer"),
        ("Fumador", "Cancer"),
        ("Cancer", "Radiografia"),
        ("Cancer", "Disnea"),
    ]
)

# Definir los parámetros (CPDs)
# La librería asigna 0 al primer valor de probabilidad y 1 al segundo: 0 con 0.9 y 1 con 0.1

cpd_pol = TabularCPD(variable="Polucion", variable_card=2, values=[[0.9], [0.1]])
cpd_fum = TabularCPD(variable="Fumador", variable_card=2, values=[[0.3], [0.7]])
cpd_cancer = TabularCPD(
    variable="Cancer",
    variable_card=2,
    values=[[0.03, 0.05, 0.001, 0.02], [0.97, 0.95, 0.999, 0.98]],
    evidence=["Fumador", "Polucion"],
    evidence_card=[2, 2],
)
cpd_rad = TabularCPD(
    variable="Radiografia",
    variable_card=2,
    values=[[0.9, 0.2], [0.1, 0.8]],
    evidence=["Cancer"],
    evidence_card=[2],
)

cpd_dis = TabularCPD(
    variable="Disnea",
    variable_card=2,
    values=[[0.65, 0.3], [0.35, 0.7]],
    evidence=["Cancer"],
    evidence_card=[2],
)

# Asociar los CPDs con la red. 
cancer_model.add_cpds(cpd_pol, cpd_fum, cpd_cancer, cpd_rad, cpd_dis)

# Comprobar si el modelo se ha construido correctamente
cancer_model.check_model()

True

In [4]:
# Acceder a los elementos del modelo
nodes = cancer_model.nodes()
edges = cancer_model.edges()
cpds = cancer_model.get_cpds()


print(f"Nodes in the model: {nodes} \n")
print(f"Edges in the model: {edges} \n")
print(f"CPDs in the model: ")
for cpd in cancer_model.get_cpds():
    print(cpd)

Nodes in the model: ['Polucion', 'Cancer', 'Fumador', 'Radiografia', 'Disnea'] 

Edges in the model: [('Polucion', 'Cancer'), ('Cancer', 'Radiografia'), ('Cancer', 'Disnea'), ('Fumador', 'Cancer')] 

CPDs in the model: 
+-------------+-----+
| Polucion(0) | 0.9 |
+-------------+-----+
| Polucion(1) | 0.1 |
+-------------+-----+
+------------+-----+
| Fumador(0) | 0.3 |
+------------+-----+
| Fumador(1) | 0.7 |
+------------+-----+
+-----------+-------------+-------------+-------------+-------------+
| Fumador   | Fumador(0)  | Fumador(0)  | Fumador(1)  | Fumador(1)  |
+-----------+-------------+-------------+-------------+-------------+
| Polucion  | Polucion(0) | Polucion(1) | Polucion(0) | Polucion(1) |
+-----------+-------------+-------------+-------------+-------------+
| Cancer(0) | 0.03        | 0.05        | 0.001       | 0.02        |
+-----------+-------------+-------------+-------------+-------------+
| Cancer(1) | 0.97        | 0.95        | 0.999       | 0.98        |
+----

In [5]:
# Inferencia por eliminación de variables 

from pgmpy.inference import VariableElimination

cancer_infer = VariableElimination(cancer_model)

# Calcular la probabilidad de Cancer dado Fumador = verdad/0

q = cancer_infer.query(variables=['Cancer'], evidence={'Fumador':0})
print(q)


# Calcular la probabilidad conjunta de Cancer y Disnea dado Fumador = falso/1
q = cancer_infer.query(variables=["Cancer", "Disnea"], evidence={"Fumador": 1})
print(q)

# Calcular las probabilidades (no conjuntas) de Cancer y Disnea dado Fumador = verdad/0
q = cancer_infer.query(variables=["Cancer", "Disnea"], evidence={"Fumador": 0}, joint=False)
for factor in q.values():
    print(factor)

+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0320 |
+-----------+---------------+
| Cancer(1) |        0.9680 |
+-----------+---------------+
+-----------+-----------+----------------------+
| Cancer    | Disnea    |   phi(Cancer,Disnea) |
+===========+===========+======================+
| Cancer(0) | Disnea(0) |               0.0019 |
+-----------+-----------+----------------------+
| Cancer(0) | Disnea(1) |               0.0010 |
+-----------+-----------+----------------------+
| Cancer(1) | Disnea(0) |               0.2991 |
+-----------+-----------+----------------------+
| Cancer(1) | Disnea(1) |               0.6980 |
+-----------+-----------+----------------------+
+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0320 |
+-----------+---------------+
| Cancer(1) |        0.9680 |
+-----------+---------------+
+-----------+---------------+
| Disnea   